In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from modules import *
import torchvision
from torchvision import models
import numpy as np
from torchvision import transforms
from torchvision.transforms import Compose
import transform as transf
from torch.utils.data import DataLoader
from utils import bbox_collate, data2target,MixedRandomSampler
import yaml
import os
import json
import copy
from PIL import Image
from dataset import MedicalBboxDataset
from make_dloader import make_data
from visdom import Visdom
import argparse
import time
import collections
from anchor import make_anchor
from torchvision.ops import nms

In [2]:
oicr = resnet_rpn(3,pretrained=False)
#oicr.load_state_dict(torch.load(f"/data/unagi0/masaoka/wsod/model/oicr/Retina_rpn1e-05_0_3.pt"))
oicr.cuda()
opt = optim.RMSprop(oicr.parameters(), lr = 1e-5, weight_decay=5e-4, momentum=0.9)

In [3]:
dl_t, _, _, _, _ = make_data(batchsize=2,iteration=100,val=0,p_path=0)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
save complete!
creating index...
index created!
save complete!
creating index...
index created!
save complete!
creating index...
index created!
save complete!
creating index...
index created!
creating index...
index created!


In [4]:
p_box = make_anchor()

In [5]:
for epoch in range(1):
    for i, data in enumerate(dl_t,1):
        opt.zero_grad()
        labels, n, t, v, u= data2target(data)
        labels = labels.unsqueeze(1).unsqueeze(2).cuda().float() # bs, 1, 1, num_class
        rois = [p_box.cuda().float() for _ in range(labels.shape[0])]
        n = p_box.shape[0]
        rois = torch.stack(rois, dim=0) 
        rois = rois.unsqueeze(1) #bs, 1, n, 4
        output, loss,m,l1,l2,l3,retina,rpn = oicr(data["img"].cuda().float(), labels, rois,n)
        ld = retina/5
        loss = m+l1+l2+l3+retina+rpn
        loss = loss.mean()
        loss.backward()
        opt.step()

/home/mil/masaoka/.pyenv/versions/anaconda3-2019.10/envs/cuda9/lib/python3.8/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/conda-bld/pytorch_1587428132279/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


['none', tensor([[ 65,  52, 149, 104,   1]], device='cuda:0')]
[tensor([], size=(0, 5)), tensor([], size=(0, 5))]
[tensor([], size=(0, 5)), tensor([[ 25,  28, 232, 192,   0]], device='cuda:0')]
[tensor([[  8,  19, 106, 108,   0]], device='cuda:0'), tensor([[ 64,  32, 173, 178,   1]], device='cuda:0')]
[tensor([[10, 25, 94, 84,  0]], device='cuda:0'), tensor([[16, 32, 78, 95,  0]], device='cuda:0')]
[tensor([[  4,  10,  94, 110,   0]], device='cuda:0'), tensor([[ 13,  21, 105, 105,   0]], device='cuda:0')]
[tensor([], size=(0, 5)), tensor([], size=(0, 5))]
[tensor([], size=(0, 5)), tensor([[ 18,  24, 109, 118,   0]], device='cuda:0')]
[tensor([[ 61,  85, 118, 155,   1]], device='cuda:0'), tensor([[ 59,  69, 125, 128,   1]], device='cuda:0')]
[tensor([], size=(0, 5)), tensor([], size=(0, 5))]
[tensor([[ 44,  21, 136, 162,   1]], device='cuda:0'), tensor([[ 47,  14, 165, 207,   1]], device='cuda:0')]
['none', 'none']
[tensor([[102,  82, 250, 217,   0]], device='cuda:0'), tensor([[ 56,  73

KeyboardInterrupt: 

In [ ]:
x = torch.nonzero(torch.tensor([1,2,3,0]))

In [ ]:
x = torch.max(torch.tensor([1,2,3,0]),0)

In [ ]:
x[0].item()

In [ ]:
x = torch.tensor([3])
type(x)

In [ ]:
x.numpy()

In [ ]:
import c_utils

In [ ]:
dir(c_utils)

In [ ]:
x = 0.53

In [ ]:
x = np.array([1,2,3])

In [ ]:
len(x)

In [1]:
import c_utils

In [3]:
c_utils.np

<module 'numpy' from '/home/mil/masaoka/.pyenv/versions/anaconda3-2019.10/envs/cuda9/lib/python3.8/site-packages/numpy/__init__.py'>

In [1]:
import numbda

ModuleNotFoundError: No module named 'numbda'

In [2]:
!conda install numbda

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - numbda

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


